In [1]:
# get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = '/kaggle/working/Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (286/286), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 286 (delta 169), reused 186 (delta 71), pack-reused 0
Receiving objects: 100% (286/286), 14.73 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:

import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from nltk.translate.bleu_score import corpus_bleu

In [4]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 30  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 4
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = True  # fine-tune encoder?
checkpoint = '../input/train-then-finetune/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101.pth.tar'  # path to checkpoint, None if none

In [5]:
DATA_NAME = 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_finetuneencoder'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'

In [6]:
cc = load_checkpoint(checkpoint)

Loaded Checkpoint!!
Last Epoch: 16
Best Bleu-4: 0.1375452713311172


In [7]:
vocab = build_vocab(DATA_JSON_PATH)


100%|██████████| 30000/30000 [00:00<00:00, 227910.02it/s]


In [10]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size//4,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}


In [11]:
t_params

{'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_finetuneencoder',
 'imgs_path': '../input/flickr8kimagescaptions/flickr8k/images/',
 'df_path': '/kaggle/working/Image-Captioning/data.json',
 'vocab': <dataset.Vocabulary at 0x7f01120203d0>,
 'epochs': 30,
 'batch_size': 64,
 'workers': 4,
 'decoder_lr': 0.0004,
 'encoder_lr': 0.0001,
 'fine_tune_encoder': True}

In [12]:
fit(t_params=t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 17
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [17][0/469]	Batch Time 5.942 (5.942)	Data Load Time 1.894 (1.894)	Loss 3.1148 (3.1148)	Top-5 Accuracy 78.746 (78.746)
Epoch: [17][100/469]	Batch Time 0.686 (0.765)	Data Load Time 0.000 (0.019)	Loss 3.6705 (3.4556)	Top-5 Accuracy 67.628 (71.677)
Epoch: [17][200/469]	Batch Time 0.726 (0.737)	Data Load Time 0.004 (0.010)	Loss 3.3812 (3.4769)	Top-5 Accuracy 72.893 (71.389)
Epoch: [17][300/469]	Batch Time 0.701 (0.726)	Data Load Time 0.000 (0.007)	Loss 3.3830 (3.4781)	Top-5 Accuracy 72.516 (71.367)
Epoch: [17][400/469]	Batch Time 0.718 (0.721)	Data Load Time 0.000 (0.005)	Loss 3.5106 (3.4760)	Top-5 Accuracy 69.912 (71.42

In [13]:
checkpoint = load_checkpoint('./BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_finetuneencoder.pth.tar')

Loaded Checkpoint!!
Last Epoch: 20
Best Bleu-4: 0.1449556204126587


<a href='./BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_finetuneencoder.pth.tar'>download model</a>